In [24]:
import pandas as pd
import numpy as np
import configparser
import matplotlib.pyplot as plt
%matplotlib inline

import re

# Importing required libraries
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM, GlobalMaxPool1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Bidirectional, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.models import Model
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras import regularizers

# gensim
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

In [4]:
config = configparser.ConfigParser()
config.read('../config.cfg')
EMBEDDING_FILE=config['FILES']['GOOGLE']
TRAIN_DATA_FILE=config['FILES']['TRAIN']
TEST_DATA_FILE=config['FILES']['TEST']

In [16]:
embed_size = 300 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [6]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [7]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [9]:
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [17]:
word_index = tokenizer.word_index
vocabulary_size=min(len(word_index)+1,max_features)

embedding_matrix = np.zeros((vocabulary_size, embed_size))


for word, i in word_index.items():
    if i>=max_features:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector    
    except KeyError:
        vec = np.zeros(embed_size)
        embedding_matrix[i]=vec

In [25]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 786s 5ms/step - loss: 0.0572 - acc: 0.9797 - val_loss: 0.0472 - val_acc: 0.9826
Epoch 2/2
143613/143613 [==============================] - 774s 5ms/step - loss: 0.0404 - acc: 0.9845 - val_loss: 0.0458 - val_acc: 0.9833


In [27]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [28]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 54s 846us/step


[0.06758899882982207, 0.9717455559372511]